In [12]:
from scipy import sparse
import pandas as pd
import os
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV
import joblib


Cargamos los datos

In [2]:
base_path = r"C:\Users\wipip\OneDrive\Documentos\GitHub\NLP-main\data\processed"

X_train = sparse.load_npz(os.path.join(base_path, "X_train.npz"))
X_test = sparse.load_npz(os.path.join(base_path, "X_test.npz"))

y_train = pd.read_csv(os.path.join(base_path, "y_train.csv"))
y_test = pd.read_csv(os.path.join(base_path, "y_test.csv"))

Declaramos el modelo SVM para el entrenamiento inicial

In [4]:
svm_model = SVC()
svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_test)

c:\Users\wipip\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Evaluamos el modelo inicial

In [5]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.955
Classification Report:
               precision    recall  f1-score   support

       False       0.95      1.00      0.97       455
        True       0.98      0.83      0.90       145

    accuracy                           0.95       600
   macro avg       0.97      0.91      0.93       600
weighted avg       0.96      0.95      0.95       600



Podemos ver que el modelo tiene un muy buen rendimiento inicial, ya que de entrada tiene una precision general de 95%, aunque podemos ver que el recall de la clase de verdaderos solo llego al 83% mientras que la de falsos fue perfecta, tenemos buen balance y precision peri vamos a intentar mejorarlo, enfocandonos en los falsos negativos

Para ello haremos un grid search para ver si logramos mejorar la precision, de no ser el caso, los resultados de presicion ya son aceptables por lo que puede quedarse de esta forma. Aun asi lo que intentaremos sera balancear para que mejore el unico punto que flaquea en las predicciones del modelo


In [6]:
svm_model = SVC(class_weight='balanced')


In [ ]:
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['linear', 'rbf', 'poly'],
    'class_weight': ['balanced', None]
}

grid_search = GridSearchCV(SVC(), param_grid, refit=True, verbose=2, cv=5)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_

In [10]:
y_pred_best = best_model.predict(X_test)

In [11]:
accuracy = accuracy_score(y_test, y_pred_best)
classification_rep = classification_report(y_test, y_pred_best)
print("Accuracy después de la optimización:", accuracy)
print("Classification Report después de la optimización:\n", classification_rep)

Accuracy después de la optimización: 0.9666666666666667
Classification Report después de la optimización:
               precision    recall  f1-score   support

       False       0.97      0.99      0.98       455
        True       0.96      0.90      0.93       145

    accuracy                           0.97       600
   macro avg       0.97      0.94      0.95       600
weighted avg       0.97      0.97      0.97       600



Despues de la optimizacion 
- Logramos mejorar la precision global.
- Mejoramos el recall de la clase de True con lo que podemos determinar que el modelo ahora ya detecta menos falsos negativos y por ende mas instancias de spam correctamente, aproximadamente un 7% lo que es una mejora bastante aceptable, esta era la oportunidad de mejora mas importante de los resultados iniciales.
- La precision de Spam verdadero disminuyo ligeramente, sin embargo el balance de recall y precision mejoro sustancialmente lo que nos da un mejor F1 score

Con estas conclusiones consideramos como exitosa la optimizacion

Guardamos el modelo y damos por concluido el proyecto 

In [13]:
ruta_modelo = r'C:\Users\wipip\OneDrive\Documentos\GitHub\NLP-main\models\svm_spam_model.pkl'
joblib.dump(best_model, ruta_modelo)

['C:\\Users\\wipip\\OneDrive\\Documentos\\GitHub\\NLP-main\\models\\svm_spam_model.pkl']